# Errors in Anisotropy Estimation

As in every image analysis pipeline, there are several possible errors that contribute to its wrong estimation. Understanding the effects of each kind of error will help us find the adequate corrections for them. We will analyze here the following sources:

- Errors in Intensity Estimation
- Errors in shift registration

## Errors in Intensity Estimation

Adequate intensity estimation requires that the images are corrected for inhomogeneities in illumination and background. What are the effects of imperfect corrections of these sources? How are these appreciated in experiments?

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

import anisotropy_functions as af
from cell_sim import Cell, Biosensor, Microscope, Corrector

In [ ]:
@interact(I_factor=(0, 2000, 100), 
          a_dimer=(0, 0.4, 0.02), a_monomer=(0, 0.4, 0.02), 
          e_par=(-1000, 1000, 50), e_per=(-1000, 1000, 50))
def plot(I_factor=1000, a_dimer=0.22, a_monomer=0.3, e_par=-400, e_per=-400):
    monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/6))
    anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, 1)
    total_fluo = af.total_fluorescence_from_monomer(monomer_fraction, 1, I_factor)
    I_parallel = af.intensity_parallel_from_anisotropy(anisotropy, total_fluo) + e_par
    I_perpendicular = af.intensity_perpendicular_from_anisotropy(anisotropy, total_fluo) + e_per
    anisotropy_from_int = af.anisotropy_from_intensity(I_parallel, 
                                                       I_perpendicular)
    
    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(5, 7))
    
    axs[0].plot(I_parallel, c='g', label='Parallel')
    axs[0].plot(I_perpendicular, c='b', label='Perpendicular')
    axs[0].plot(I_parallel - e_par, c='g', alpha=0.5, label='Real Parallel')
    axs[0].plot(I_perpendicular - e_per, c='b', alpha=0.5, label='Real Perpendicular')
    axs[0].legend()
    axs[0].set_ylabel('Intensity (u.a.)')
    
    axs[1].axhline(y=a_dimer, color='k', ls='--')
    axs[1].axhline(y=a_monomer, color='k', ls='--')
    axs[1].plot(anisotropy_from_int, color='r', label='Estimated')
    axs[1].plot(anisotropy, color='r', alpha=0.5, label='Real')
    axs[1].set_ylabel('Anisotropy')
    axs[1].legend()

    plt.subplots_adjust(hspace=0)
    plt.show()

It is important to notice that there different monomer and dimer anisotropies are possible depending on the errors. We must highlight that in some cases monomer anisotropy might be higher than dimer anisotropy, and this looks like a reversed anisotropy curve. Aditionally, theoritacally impossible values are also possible depending on the magnitude of the errors. Furthermore, if the error in intensity has a time dependance this will affect the shape of the curve. 

## Errors in shift registration

Due to the thickness of high quality polarizers, it is likely that parallel and perpendicular images are shifted between each other. If we were to generate an anisotropy image, we need to be able to correct this shift. What happens if this is not adequately corrected? How can we bypass these problems? How can we estimate the best correction?

Let's begin by generating a simulated squared cell with a gradient of concentration of biosensors. We can choose the maximum number of biosensors expected and if we are to add poisson noise to this number.

In [ ]:
proteins = 800
poisson = True

cell = Cell(proteins, poisson)

In [ ]:
plt.imshow(cell.cell_image, interpolation='none')
plt.colorbar()
plt.axis('off')
plt.show()

Let's define an anisotropy state for the cell.

In [ ]:
anisotropy = 0.26
cell.add_biosensor({'anisotropy_monomer':0.3, 'anisotropy_dimer': 0.22, 'delta_b': 0.15})

cell.generate_intensity_images(anisotropy)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()

this_im = axs[0].imshow(cell.parallel_image, interpolation='none')
fig.colorbar(this_im, ax=axs[0])

axs[0].axis('off')
axs[0].set_title('Parallel Image')

this_im = axs[1].imshow(cell.perpendicular_image, interpolation='none')
fig.colorbar(this_im, ax=axs[1])

axs[1].axis('off')
axs[1].set_title('Perpendicular Image')

axs[2].hist(cell.parallel_image[200:800, 200:800].flatten(), bins=100, log=True)
axs[2].set_title('Parallel Image Histogram')

axs[3].hist(cell.perpendicular_image[200:800, 200:800].flatten(), bins=100, log=True)
axs[3].set_title('Perpendicular Image Histogram')

plt.show()

We should estimate now the anisotropy image before adding acquisition noise.

In [ ]:
non_acquired_anisotropy_image = np.zeros_like(cell.parallel_image)
nonzeros = cell.mask
non_acquired_anisotropy_image[nonzeros] = af.anisotropy_from_intensity(cell.parallel_image[nonzeros], cell.perpendicular_image[nonzeros])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(non_acquired_anisotropy_image, vmin=0.19, vmax=0.4, interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(non_acquired_anisotropy_image[cell.mask].flatten(), bins=np.arange(0.2, 0.3, 0.001))
axs[1].axvline(x=anisotropy, color='k', ls='--')

plt.show()

After obtaining both intensity images, we could add some aquisition noise to the images.

In [ ]:
microscope = Microscope()

parallel_image, perpendicular_image = microscope.acquire_cell(cell)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()

this_im = axs[0].imshow(parallel_image, interpolation='none')
fig.colorbar(this_im, ax=axs[0])

axs[0].axis('off')
axs[0].set_title('Parallel Image')

this_im = axs[1].imshow(perpendicular_image, interpolation='none')
fig.colorbar(this_im, ax=axs[1])

axs[1].axis('off')
axs[1].set_title('Perpendicular Image')

axs[2].hist(parallel_image.flatten(), bins=100, log=True)
axs[2].set_title('Parallel Image Histogram')

axs[3].hist(perpendicular_image.flatten(), bins=100, log=True)
axs[3].set_title('Perpendicular Image Histogram')

plt.show()

In [ ]:
anisotropy_image = np.zeros_like(parallel_image)
nonzeros = cell.mask
anisotropy_image[nonzeros] = af.anisotropy_from_intensity(parallel_image[nonzeros], perpendicular_image[nonzeros])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(anisotropy_image, interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(anisotropy_image[cell.mask].flatten(), bins=100)
axs[1].axvline(x=anisotropy, color='k', ls='--')

plt.show()

Now we need to add the image analysis steps and corrections we would normally implement to test them and choose the best option.

In [ ]:
corrector = Corrector()
corrected_parallel, corrected_perpendicular = corrector.correct(parallel_image, perpendicular_image)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()

this_im = axs[0].imshow(corrected_parallel, interpolation='none')
fig.colorbar(this_im, ax=axs[0])

axs[0].axis('off')
axs[0].set_title('Parallel Image')

this_im = axs[1].imshow(corrected_perpendicular, interpolation='none')
fig.colorbar(this_im, ax=axs[1])

axs[1].axis('off')
axs[1].set_title('Perpendicular Image')

axs[2].hist(corrected_parallel.flatten(), bins=100, log=True)
axs[2].set_title('Parallel Image Histogram')

axs[3].hist(corrected_perpendicular.flatten(), bins=100, log=True)
axs[3].set_title('Perpendicular Image Histogram')

plt.show()

In [ ]:
corrected_anisotropy_image = np.zeros_like(corrected_parallel)
nonzeros = np.nonzero(corrected_parallel)
corrected_anisotropy_image[nonzeros] = af.anisotropy_from_intensity(corrected_parallel[nonzeros], corrected_perpendicular[nonzeros])

We should shrink the mask to avoid border low intensity and noisy pixels.

In [ ]:
def shrink_mask(mask, shrink=0.2):
    inds = np.where(cell.mask[cell.mask.shape[0] //2])[0]
    ini, end = inds[np.ceil(shrink / 2 * len(inds)).astype(int)], inds[np.floor((1 - shrink / 2) * len(inds)).astype(int)]

    mask = np.zeros_like(corrected_anisotropy_image).astype(bool)
    mask[ini:end, ini:end] = True
    return mask

In [ ]:
mask = cell.mask.copy()
mask = shrink_mask(mask, 0.2)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

this_im = axs[0].imshow(corrected_anisotropy_image, cmap='plasma', vmin=0.15, vmax=0.3, interpolation='none')
axs[0].axis('off')
fig.colorbar(this_im, ax=axs[0])

axs[1].hist(corrected_anisotropy_image[mask].flatten(), bins=np.arange(0.1, 0.45, 0.002), log=True)
axs[1].axvline(x=anisotropy, color='k', ls='--')

plt.show()

Up to here looks like the correction worked and generated a good anisotropy image.

What would happen if our shift correction was not perfect? What happens if our background correction is not perfect?

In [ ]:
@interact(x_shift=(-20, 20, 1), y_shift=(-20, 20, 1), bkg_value=(-20, 20, 1))
def plot(x_shift=0, y_shift=0, bkg_value=0):
    imperfect_corrector = Corrector()
    imperfect_corrector.bkg_params ={'bkg_value': 200 + bkg_value}
    imperfect_corrector.shift = (-4 + x_shift, -6 + y_shift)
    corrected_parallel, corrected_perpendicular = imperfect_corrector.correct(parallel_image, perpendicular_image)
    
    corrected_anisotropy_image = np.zeros_like(corrected_parallel)
    nonzeros = np.nonzero(corrected_parallel)
    corrected_anisotropy_image[nonzeros] = af.anisotropy_from_intensity(corrected_parallel[nonzeros], corrected_perpendicular[nonzeros])
    
    mask = cell.mask.copy()
    mask = shrink_mask(mask, 0.2)
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))

    this_im = axs[0].imshow(corrected_anisotropy_image, cmap='plasma', vmin=0.15, vmax=0.3, interpolation='none')
    axs[0].axis('off')
    fig.colorbar(this_im, ax=axs[0])

    axs[1].hist(corrected_anisotropy_image[mask].flatten(), bins=np.arange(0.1, 0.45, 0.002), log=True)
    axs[1].axvline(x=anisotropy, color='k', ls='--')

    plt.show()
    
    print('Mean: %0.6f; STD: %0.6f' % (np.mean(corrected_anisotropy_image[mask].flatten()), 
                                       np.std(corrected_anisotropy_image[mask].flatten())))

## $\Delta$b Parameter

To understand the effect of this parameter, we need to simulate a curve of the fraction of monomers and translate it into anisotropy. As far as we know, deriving the curve will yield a curve proportionate to complex or, in a way, activity per unit of time.

In [ ]:
rate = 6

monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/rate))
monomer_derivative = np.diff(monomer_fraction)
time_maximum_activity = np.where(monomer_derivative == np.max(monomer_derivative))[0][0]
time = np.arange(0, len(monomer_fraction))

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(5, 5))

axs[0].plot(time, monomer_fraction)

axs[1].axvline(x=time_maximum_activity, color='k', alpha=0.7, linestyle='--')
axs[1].plot(time[:-1], np.diff(monomer_fraction))

axs[0].set_ylabel('Anisotropy')
axs[1].set_ylabel('Monomer Fraction\nDerivative')
axs[1].set_xlabel('Time (min.)')

axs[1].set_yticks([])

plt.subplots_adjust(hspace=0)
plt.show()

From the monomer curve we can estimate the anisotropy curves by means of $\Delta$b parameter as well as monomer and dimer anisotropy.

In [ ]:
a_dimer=0.22
a_monomer=0.3
delta_b = 0.15
@interact(a_dimer=(0.18, 0.33, 0.01), a_monomer=(0.18, 0.33, 0.01), delta_b=(-0.7, 0.8, 0.1))
def plot(a_dimer=0.22, a_monomer=0.3, delta_b = 0.15):
    b = 1 + delta_b

    anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, b)
    der_ani = np.diff(anisotropy)
    time_maximum_derivative = np.where(der_ani == np.max(der_ani))[0][0]

    fig, axs = plt.subplots(3, 1, sharex=True, figsize=(5, 8))
    axs[0].plot(anisotropy, color='r')

    anisotropy_normalized = anisotropy[:-1] - np.min(anisotropy)
    anisotropy_normalized = anisotropy_normalized / np.max(anisotropy_normalized)
    activity = der_ani / ((1 + (b-1) * anisotropy_normalized) ** 2)
    
    time_maximum_activity_obs = np.where(activity == np.max(activity))[0][0]

    axs[1].plot(der_ani / np.max(der_ani), color='r')

    axs[2].plot(activity / np.max(activity), color='r')

    axs[0].axhline(y=a_dimer, color='k', ls='--')
    axs[0].axhline(y=a_monomer, color='k', ls='--')
    
    axs[1].axvline(x=time_maximum_derivative, color='k', ls='--', alpha=0.7)
    axs[2].axvline(x=time_maximum_activity_obs, color='k', ls='--', alpha=0.7)
    
    axs[1].axhline(y=0, color='k')
    axs[2].axhline(y=0, color='k')

    axs[0].set_ylabel('Anisotropy')
    axs[1].set_ylabel('Anisotropy Derivative')
    axs[2].set_ylabel('Activity')
    axs[2].set_xlabel('Time (min.)')
    plt.subplots_adjust(hspace=0)
    plt.show()
    
    print('Anisotropy Time: %d; Activity Time: %d' % (time_maximum_derivative, time_maximum_activity_obs))

As calculating the derivative of an experimental curve is an ill-posed problem, we should assess our method to calculate its derivative. Let's define the functions we are to use to find the time of maximum activity. We should also evaluate its behaviour with noise.

In [ ]:
from scipy.signal import savgol_filter
from scipy.interpolate import splrep, splev


def calculate_activity(ani, time_step, delta_b=0):
    """Uses at least 3 points or 20 minutes to estimate the derivative of the 
    curve using Savitzky-Golay Filter."""
    window_length = int(np.floor(20 / time_step))
    if window_length % 2 == 0:
        window_length += 1
    window_length = np.max((window_length, 3))
    
    der = savgol_filter(ani, window_length=window_length,
                        polyorder=2,
                        deriv=1, delta=time_step, mode='nearest')

    anisotropy_normalized = ani - np.min(ani)
    anisotropy_normalized = anisotropy_normalized / np.max(anisotropy_normalized)
    activity = der / ((1 + delta_b * anisotropy_normalized) ** 2)
    return activity

def interpolate(new_time, time, curve):
    """Interpolate curve using new_time as xdata"""
    if not np.isfinite(time).all():
        return np.array([np.nan])

    f = splrep(time, curve, k=3)
    return splev(new_time, f, der=0)

In [ ]:
@interact(N=(1, 4, 1), time_resolution=(1, 20, 1), experimental_noise=(0, 0.05, 0.001), delta_b=(-0.7, 0.8, 0.1))
def plot(N=2, time_resolution=5, experimental_noise=0.003, delta_b=0):
    N = 10 ** N
    monomer_fraction = 1 / (1 + np.exp(-np.arange(-50, 50)/6))
    anisotropy = af.anisotropy_from_monomer(monomer_fraction, a_monomer, a_dimer, 1 + delta_b)
    anisotropy_spaced = anisotropy[::time_resolution]
    time_spaced = time[::time_resolution]

    fig, axs = plt.subplots(3, 1, sharex=True, figsize=(3, 6))
    time_maxs = []
    for i in range(N):
        anisotropy_exp = anisotropy_spaced + np.random.normal(0, experimental_noise, len(anisotropy_spaced))
        activity_exp = calculate_activity(anisotropy_exp, time_step=time_resolution, delta_b=delta_b)
        activity_interp = interpolate(time, time_spaced, activity_exp)
        time_maxs.append(np.where(activity_interp[:-10] == np.max(activity_interp[:-10]))[0][0])

        axs[0].plot(time_spaced, anisotropy_exp, alpha=0.3)

        axs[1].plot(time[:-10], activity_interp[:-10], alpha=0.3)
        
    axs[0].plot(time, anisotropy, color='k', linewidth=3)
    axs[0].plot(time, anisotropy, color='r')

    activity_real = calculate_activity(anisotropy, time_step=1, delta_b=delta_b)
    axs[1].plot(time, activity_real, color='k', linewidth=3)
    axs[1].plot(time, activity_real, color='r')
    
    time_maximum_activity_real = np.where(activity_real == np.max(activity_real))[0][0]
    axs[2].axvline(x=time_maximum_activity_real, color='k', alpha=0.7, linestyle='--')
    axs[2].hist(time_maxs, bins=time)

    axs[0].set_ylabel('Anisotropy')
    axs[1].set_ylabel('Activity')
    axs[2].set_ylabel('Counts')
    axs[2].set_xlabel('Time (min.)')

    axs[0].set_yticks([0.2, 0.25, 0.3])
    axs[1].set_yticks([])

    plt.subplots_adjust(hspace=0)
    plt.show()
    print('Time of Maximum Activity: %0.1f \pm %0.1f' % (np.mean(time_maxs), np.std(time_maxs)))